In [1]:
from bs4 import BeautifulSoup
from dateutil.parser import parse

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import time

import sys

firefox_capabilities = DesiredCapabilities.FIREFOX
firefox_capabilities['marionette'] = True
firefox_capabilities['handleAlerts'] = True
firefox_capabilities['acceptSslCerts'] = True
firefox_capabilities['acceptInsecureCerts'] = True
geckoPath = 'driver/geckodriver.exe'
#capabilities=firefox_capabilities,
firefox = webdriver.Firefox(executable_path=geckoPath)
#driver = webdriver.PhantomJS(executable_path='driver/phantomjs.exe')
driver = webdriver.Firefox( executable_path=geckoPath)
try:
    firefox.get('https://www.crunchbase.com/organization/apple')
except:
    pass
driver.get('https://techcrunch.com/')
firefox.set_page_load_timeout(2)
driver.set_page_load_timeout(30)

def existence_in_crunchbase(name):
    base_url = 'www.crunchbase.com'
    url = 'https://techcrunch.com/search/'+name
    url_crunchbase = None
    try:
        driver.get(url)
    except KeyboardInterrupt:
        sys.exit()
    except:
        print('timeout of phantomjs')
        pass
    
    soup = BeautifulSoup(driver.page_source, "html.parser")
    blocks = soup.select("h2.post-title a")
    if len(blocks)>0:
        url_crunchbase = blocks[0]["href"]  
        name  = blocks[0].getText()
 
        if base_url not in url_crunchbase:
            url_crunchbase = None
    
    return url_crunchbase
            

def extract_company_techcrunch(name,url):
    try:
        firefox.get(url)
    except:
        pass
    founded = None
    employees = None
    company_name = ''
    blocks_dt = []
    blocks_dd = []
    try:
        soup = BeautifulSoup(firefox.page_source, "html.parser")
        blocks_dt = soup.select("div.details dt")
        blocks_dd = soup.select("div.details dd")
        company_name_tag = soup.select_one("#profile_header_heading")
        if company_name_tag == None:
            company_name = ''
        else:
            company_name = company_name_tag.getText()
    except:
        pass

    i = 0
    while company_name.lower() != name.lower() or len(blocks_dt)==0 :
        time.sleep(0.3)
        try:
            soup = BeautifulSoup(firefox.page_source, "html.parser")
            company_name_tag = soup.select_one("#profile_header_heading")
            if company_name_tag:
                company_name = company_name_tag.getText()
            else:
                company_name = ''
            blocks_dt = soup.select("div.details dt")
            blocks_dd = soup.select("div.details dd")
        except KeyboardInterrupt:
            sys.exit()
        except:
            pass
        i=i+1
        if i >= 100:
            print('fail to crawl ', name , ' in crunchbase')
            break

    
    if company_name.lower() == name.lower():
        for index,block in enumerate(blocks_dt):
            if 'Founded' in  block.getText() : 
                founded = blocks_dd[index].getText()
            if 'Employees' in block.getText():
                employees = blocks_dd[index].getText().split('|')[0]
    print( name , founded , employees)
    return founded,employees



In [4]:
import time
import json
import os

class CompaniesManager:
    def __init__(self):
        self.companies = []
        self.articles = []
        self.companies_name = []
        self.load_articles()
        self.load_companies()
        
    def load_articles(self):
        if os.path.isfile('data/raw_articles.json'):
            f = open('data/raw_articles.json')
            self.articles = json.load(f)
            f.close()
            
    def load_companies(self):
        if os.path.isfile('data/raw_companies.json'):
            f = open('data/raw_companies.json')
            self.companies = json.load(f)
            f.close()      
            for company in self.companies:
                name = company["name"]
                self.companies_name.append(name)
        
            
    def extract_companies(self):
        
        for index,article in enumerate(self.articles):
            for company_name in article["companies"]: 
                article_extraInfos = article["extra_infos"]
                for info in article_extraInfos:
                    if info["text"]== company_name:
                        relevance = info["relevance"]
                        count_in_article = info["count"]
                            
                if company_name in self.companies_name :
                    for company in self.companies:
                        if company_name == company["name"] and article["id"] not in company["articles"]: 
                            company["count"] = company["count"]+1
                            company["sentiment"]= company["sentiment"]+article["sentiment"]
                            company["articles"].append(article["id"])
                            extra_infos = {
                                 "id":article["id"] ,
                                 "count_in_article":count_in_article,
                                 "revelance": relevance       
                            }
                            company["extra_infos"].append(extra_infos)
                            
                else:
                    company = {
                        "name": company_name,
                        "dateFound": int(str(time.time()).split('.')[0]),
                        "count":1,
                        "sentiment": article["sentiment"],
                        "articles": [article["id"]],
                        "extra_infos":
                        [
                            {
                             "id": article["id"],
                             "count_in_article":count_in_article,
                             "revelance": relevance
                            }  
                        ]    
                    }
                    self.companies.append(company)
                    self.companies_name.append(company_name)
            
        self.save_to_disk()
        
    def extend_crunch(self):
        
        i = 0
        for index,company in enumerate(self.companies):
            name = company["name"]
            search_label = None
            if "search_label" in company:
                search_label = company["search_label"]
                
            if search_label==None :
                url_crunchbase = existence_in_crunchbase(name)
                
                if url_crunchbase != None:
                    print(name, ' in crunchbase')
                    i=i+1
                    founded,employees = extract_company_techcrunch(name,url_crunchbase)
                    company["search_label"] = url_crunchbase
                    if founded is not None:
                        company["foundationDate"] = founded
                    if employees is not None:
                        company["number_of_employees"] = employees
                    
                else:
                    company["search_label"] = str(0)
                    print(name, 'not in crunchbase')
            else:
                print("skip: ", name)
            
            if (i+1)%100 == 0:
                self.save_to_disk()
        self.save_to_disk()            
                                
    def save_to_disk(self):
        with open('data/raw_companies.json', 'w') as company_file:
            json.dump(self.companies, company_file,indent = 2)
            company_file.close()
                
                
                
            
    

In [5]:
CM = CompaniesManager()
#CM.extract_companies()
CM.extend_crunch()
CM.save_to_disk()

skip:  SENSORO
skip:  Microsoft Corp
skip:  Sumitomo
skip:  Manchester City Verve
skip:  SalesWings
skip:  SA tech
skip:  Avito
skip:  Tshimologong Innovation Hub
skip:  Flickr
skip:  Niantic Labs
skip:  Oblong
skip:  Osterhout Design Group
skip:  Niantic
skip:  Oblong Industries
skip:  Mezzanine
skip:  The Pokemon Company
skip:  GamesBeat
skip:  BNJ Marketing
skip:  DormaKaba
skip:  Berkshire companies
skip:  Bezos
skip:  Private Equity
skip:  Utrip
skip:  Greylock
skip:  Elsight
skip:  LiveU
skip:  WAN
skip:  Vodaphone
skip:  Optus
skip:  GoRemote
skip:  Agri Alliance
skip:  Bitbond
skip:  Obotritia
skip:  FiveAI
skip:  GuestReady
skip:  Carspring
skip:  Moneybox
skip:  Drivetribe
skip:  Realla
skip:  Habito
skip:  Sling & Stone
skip:  Crowdcube
skip:  Cali Rice
skip:  Nominet Trust
skip:  3D Robotics
skip:  AgFunder
skip:  Airware
skip:  Arbe Robotics
skip:  Zipline International
skip:  Redbird
skip:  Bloomberg Businessweek
skip:  Yuneec Electric Aviation
skip:  CyPhy Works
skip:  E

skip:  Imagination Technologies
skip:  Rambus
skip:  SiFive
skip:  Citymapper
skip:  Microsemi
skip:  Altera
skip:  Cadence Design Systems
skip:  United Design Systems
skip:  Linley Group
skip:  Synopsys
skip:  Atmel
skip:  HP Enterprise
skip:  Juniper
skip:  livestreaming
skip:  Deezer
skip:  Ultra
skip:  Goldenvoice
skip:  fiat
skip:  PST Consulting
skip:  TGPR
skip:  NOLs
skip:  JumpStart Inc.
skip:  Fog Creek Software
skip:  Allied Gear
skip:  Internet Corporation
skip:  Continuum Analytics
skip:  Sertain Research
skip:  MIT Media Lab
skip:  StreamFuzion Corp
skip:  Bot Insight
skip:  Le Taxi
skip:  Credit Karma
skip:  AIstartups
skip:  Humu
skip:  Greyball
skip:  Shearman & Sterling LLP
skip:  Shearman
skip:  DeepMap
skip:  TetraVue
skip:  Cruise Automation
skip:  Silicon Valley-based
skip:  Proofpoint
skip:  FireLayers
skip:  Twistlock
skip:  Omniture
skip:  Fastdata.io
skip:  FDIO Engine
skip:  Abundant Robotics
skip:  Abundant Robotic
skip:  GV
skip:  Yamaha Motor Company
skip:

skip:  Homecamp
skip:  Super Retail Group
skip:  Youcamp
skip:  Kaplan
skip:  Red Marker
skip:  BeforeYouBid
skip:  Creel Price
skip:  QSR
skip:  Nuance
skip:  Quick Service Restaurants
skip:  Makers Empire
skip:  Adelaide startup Makers Empire
skip:  DIT
skip:  DIT Australian
skip:  Redballoon
skip:  Yume
skip:  Norwegian Salmon
skip:  Impact Generation Partners
skip:  PacWest
skip:  AFR
skip:  Pepper Money
skip:  Reef Innovation Fund
skip:  Baryamal
skip:  theSPACE Australasia
skip:  Gold Coast
skip:  iSelect
skip:  Air Events Global
skip:  Ericsson Australia
skip:  Campbell Scientific Australia
skip:  Sierra Wireless
skip:  Pooley Wines
skip:  Campbell Scientific
skip:  RocketSpace
skip:  RefLIVE
skip:  Tablo
skip:  Tablo Communities
skip:  Westpac
skip:  Rendevu
skip:  Bardi
skip:  HerePin
skip:  Travello
skip:  DoTalk
skip:  Doppler Labs
skip:  Prospa
skip:  Spacecubed
skip:  Seven West Media
skip:  HappyCo
skip:  Formalytics
skip:  Busybot
skip:  GoRepp
skip:  Investec Australia


skip:  Volusion
skip:  Infiniscene
skip:  Instinet
skip:  BusinessCollective
skip:  CHASE
skip:  JUGGLE
skip:  Traackr
skip:  BuzzSumo
skip:  Followerwonk
skip:  Moz
skip:  Logicalis Healthcare Solutions
skip:  ezLandlordForms
skip:  Vedsutra
skip:  European Circuits
skip:  Pando
skip:  LeapLearner
skip:  LeapLearner China
skip:  Expert Market
skip:  LearnLaunch
skip:  Entri
skip:  FinLab
skip:  Chicago Super Sitters
skip:  OneTable
skip:  OptinMonster
skip:  Dog Patch Labs
skip:  Cesanta
skip:  Instagrammers
skip:  JWT
skip:  Tech.Co
skip:  Custom Fit Sandals
skip:  Optimizilla
skip:  Squarespace
skip:  SumoMe
skip:  Wix
skip:  CTA Digital NYC
skip:  Voices.com
skip:  LMB Group
skip:  Movable Ink
skip:  Shoppable
skip:  Relevancystacking.com
skip:  Cleverbridge
skip:  Simple Mills
skip:  Whole Foods
skip:  Mariono
skip:  Pika Energy
skip:  Wallaroo Media
skip:  Dabble
skip:  Slack Care
skip:  LiquidSpace
skip:  Queensbridge Public Housing
skip:  OKCupid
skip:  Savills
skip:  BlendScor

skip:  DOT
skip:  Stephens Inc
skip:  AI technologies
skip:  ClearMetal
skip:  Signifyd
skip:  Prevedere
skip:  Illusive Networks
skip:  Infocyte
skip:  MindMeld
skip:  USAA
skip:  Uniqlo
skip:  IDG
skip:  MapD
skip:  Nectar Labs
skip:  Dosa Restaurant
skip:  Chegg
skip:  Conde Nast
skip:  Genwi
skip:  Xamarin
skip:  Guardant Health
skip:  Guardant
skip:  Temasek
skip:  SpyBiotech
skip:  Lighthouse
skip:  Ontruck
skip:  Idinvest
skip:  Design Inc.
skip:  Harrison Metal
skip:  Deep Instinct
skip:  Athelas
skip:  Focal Systems
skip:  Fidelity Investments
skip:  Genetesis
skip:  Cylance
skip:  Bay Labs
skip:  Harman Kardon Invoke
skip:  LivePerson
skip:  Azure App Services
skip:  Razor Pages
skip:  Cockroach Labs
skip:  CockroachDB Enterprise
skip:  Mya Systems
skip:  Teamable
skip:  Overstock.com
skip:  ARK Invest
skip:  Execution Labs
skip:  Small Business Owners
skip:  TRANSACT
skip:  Global Payments Inc.
skip:  Dunkin' Donuts
skip:  Cheesecake Factory
skip:  NYSE
skip:  Pivotal
skip: 

MessageLabs  in crunchbase
MessageLabs 1999 None found in Crunchbase
Cisco Investments  in crunchbase
Cisco Investments Unknown None
Akili Interactive Labs  in crunchbase
Akili Interactive Labs 2011 11 - 50 
Listen.com not in crunchbase
Real Networks  in crunchbase
Real Networks 1995 501 - 1k 
Neuroscape not in crunchbase
Phlutter not in crunchbase
Accern  in crunchbase
Accern June 13, 2013 1 - 10 
United  in crunchbase
United 1988 251 - 500 
Affinity Investment Group  in crunchbase
Affinity Investment Group Unknown None found in Crunchbase
Bagel Labs  in crunchbase
Bagel Labs January 25, 2016 1 - 10 
Shyft Technologies  in crunchbase
Shyft Technologies July 1, 2015 51 - 100 
Donut Media  in crunchbase
Donut Media July 1, 2015 1 - 10 
Blue River Technology  in crunchbase
Blue River Technology 2011 11 - 50 
Latin America not in crunchbase
BR Startups not in crunchbase
LAVCA  in crunchbase
LAVCA May 12, 2009 1 - 10 
QueroQuitar not in crunchbase
Finnovista  in crunchbase
Finnovista Unkno